In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score,classification_report
import imblearn
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

In [17]:
df=pd.read_csv(r"C:\Users\prashant\Downloads\train_ctrUa4K (1).csv")

In [18]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [19]:
df.drop(['Loan_ID'],axis=1,inplace=True)

In [20]:
df.isnull().sum()

Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [21]:
df['Credit_History'].value_counts()

1.0    475
0.0     89
Name: Credit_History, dtype: int64

In [22]:
for i in ['LoanAmount','Loan_Amount_Term']:
    df[i].replace(0,np.nan,inplace=True)
    df[i].fillna(df[i].mean(),inplace=True)

In [23]:
df.dropna(inplace=True)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 511 entries, 0 to 613
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             511 non-null    object 
 1   Married            511 non-null    object 
 2   Dependents         511 non-null    object 
 3   Education          511 non-null    object 
 4   Self_Employed      511 non-null    object 
 5   ApplicantIncome    511 non-null    int64  
 6   CoapplicantIncome  511 non-null    float64
 7   LoanAmount         511 non-null    float64
 8   Loan_Amount_Term   511 non-null    float64
 9   Credit_History     511 non-null    float64
 10  Property_Area      511 non-null    object 
 11  Loan_Status        511 non-null    object 
dtypes: float64(4), int64(1), object(7)
memory usage: 51.9+ KB


In [25]:
categorical_columns = []
numeric_columns = []

for col in df.columns:
    if df[col].dtype == 'object':
        categorical_columns.append(col)
    else:
        numeric_columns.append(col)

print("Categorical columns:", categorical_columns)
print("Numeric columns:", numeric_columns)

Categorical columns: ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area', 'Loan_Status']
Numeric columns: ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']


In [26]:
df[numeric_columns].corr()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
ApplicantIncome,1.000000,-0.114302,0.490153,-0.007649,-0.044954
CoapplicantIncome,-0.114302,1.000000,0.192269,-0.008529,0.000391
LoanAmount,0.490153,0.192269,1.000000,0.049991,-0.027274
Loan_Amount_Term,-0.007649,-0.008529,0.049991,1.000000,0.024302
Credit_History,-0.044954,0.000391,-0.027274,0.024302,1.000000


In [27]:
for col in categorical_columns:
    df[col] = LabelEncoder().fit_transform(df[col])

In [28]:
scaler = StandardScaler()
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

In [34]:
df.corr()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
Gender,1.000000,0.363499,0.200692,0.044667,-0.007948,0.035377,0.157993,0.102137,-0.088109,0.024682,-0.020576,0.052533
Married,0.363499,1.000000,0.375597,-0.002516,0.019750,0.040226,0.107327,0.181882,-0.110044,0.020519,0.029479,0.098560
Dependents,0.200692,0.375597,1.000000,0.039250,0.051180,0.124717,-0.010586,0.168145,-0.099282,-0.039492,-0.000748,0.016984
Education,0.044667,-0.002516,0.039250,1.000000,-0.014796,-0.142747,-0.067085,-0.171444,-0.112531,-0.075720,-0.033095,-0.088699
Self_Employed,-0.007948,0.019750,0.051180,-0.014796,1.000000,0.167491,-0.012554,0.115941,-0.028340,-0.016306,-0.052259,-0.018705
ApplicantIncome,0.035377,0.040226,0.124717,-0.142747,0.167491,1.000000,-0.114302,0.490153,-0.007649,-0.044954,-0.066311,-0.042166
CoapplicantIncome,0.157993,0.107327,-0.010586,-0.067085,-0.012554,-0.114302,1.000000,0.192269,-0.008529,0.000391,0.000368,-0.039323
LoanAmount,0.102137,0.181882,0.168145,-0.171444,0.115941,0.490153,0.192269,1.000000,0.049991,-0.027274,-0.113254,-0.062882
Loan_Amount_Term,-0.088109,-0.110044,-0.099282,-0.112531,-0.028340,-0.007649,-0.008529,0.049991,1.000000,0.024302,-0.067434,0.004054
Credit_History,0.024682,0.020519,-0.039492,-0.075720,-0.016306,-0.044954,0.000391,-0.027274,0.024302,1.000000,-0.003404,0.545934


In [29]:
X = df.drop('Loan_Status', axis=1)  # Features
y = df['Loan_Status']  # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [30]:
models = [
        ('Logistic Regression', LogisticRegression()),
        ('SVM', SVC()),
        ('KNN', KNeighborsClassifier()),
        ('Decision Tree', DecisionTreeClassifier())
    ]
    
for name, model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        print(f"Model: {name}")
        print("Accuracy:", accuracy_score(y_test, y_pred))
        print("Classification Report:\n", classification_report(y_test, y_pred))

Model: Logistic Regression
Accuracy: 0.8181818181818182
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.52      0.66        52
           1       0.80      0.97      0.88       102

    accuracy                           0.82       154
   macro avg       0.85      0.74      0.77       154
weighted avg       0.83      0.82      0.80       154

Model: SVM
Accuracy: 0.8051948051948052
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.50      0.63        52
           1       0.79      0.96      0.87       102

    accuracy                           0.81       154
   macro avg       0.83      0.73      0.75       154
weighted avg       0.82      0.81      0.79       154

Model: KNN
Accuracy: 0.7792207792207793
Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.50      0.60        52
           1       0.78      

In [38]:
models = [
        ('Logistic Regression', LogisticRegression()),
        ('SVM', SVC()),
        ('KNN', KNeighborsClassifier()),
        ('Decision Tree', DecisionTreeClassifier())
    ]
    
for name, model in models:
        model.fit(X_train, y_train)
        y_pred_train = model.predict(X_train)
        
        print(f"Model: {name}")
        print("Accuracy:", accuracy_score(y_train,y_pred_train))
        print("Classification Report:\n", classification_report(y_train,y_pred_train))

Model: Logistic Regression
Accuracy: 0.8067226890756303
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.43      0.58       112
           1       0.79      0.98      0.87       245

    accuracy                           0.81       357
   macro avg       0.85      0.70      0.73       357
weighted avg       0.83      0.81      0.78       357

Model: SVM
Accuracy: 0.8291316526610645
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.48      0.64       112
           1       0.81      0.99      0.89       245

    accuracy                           0.83       357
   macro avg       0.88      0.73      0.76       357
weighted avg       0.85      0.83      0.81       357

Model: KNN
Accuracy: 0.8235294117647058
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.55      0.66       112
           1       0.82      

In [33]:
# Model Selection and Hyperparameter Tuning
best_model = None
best_accuracy = 0.0
best_params = None
models = [('Logistic Regression', LogisticRegression(), {'C': [0.001, 0.01, 0.1, 1, 10], 
                                                       'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                                                       'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
                                                       'max_iter': [100, 200, 300]}),
        ('SVM', SVC(), {'C': [0.1, 1, 10], 
                        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 
                        'gamma': ['scale', 'auto']}),
        ('KNN', KNeighborsClassifier(), {'n_neighbors': [3, 5, 7],
                                         'weights': ['uniform', 'distance'],
                                         'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                                         'p': [1, 2]}),
        ('Decision Tree', DecisionTreeClassifier(), {'max_depth': [None, 10, 20], 
                                                     'min_samples_split': [2, 5, 10]})

    ]
for name, model, param_grid in models:
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train,y_train)
        
    y_pred = grid_search.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
        
    print(f"Model: {name}")
    print("Best Parameters:", grid_search.best_params_)
    print("Accuracy:", accuracy)
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("="*50)
        
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = name
        best_params = grid_search.best_params_
    
print("Best Model:", best_model)
print("Best Parameters:", best_params)
print("Best Accuracy:", best_accuracy)

Model: Logistic Regression
Best Parameters: {'C': 0.001, 'max_iter': 100, 'penalty': 'none', 'solver': 'newton-cg'}
Accuracy: 0.8181818181818182
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.52      0.66        52
           1       0.80      0.97      0.88       102

    accuracy                           0.82       154
   macro avg       0.85      0.74      0.77       154
weighted avg       0.83      0.82      0.80       154

Model: SVM
Best Parameters: {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Accuracy: 0.8181818181818182
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.52      0.66        52
           1       0.80      0.97      0.88       102

    accuracy                           0.82       154
   macro avg       0.85      0.74      0.77       154
weighted avg       0.83      0.82      0.80       154

Model: KNN
Best Parameters: {'algorithm': 'auto